In [2]:
import csv
import logging
import random
import time
import os
from datetime import datetime

# --- 1. CONFIGURATION ---
LOG_FILE = "hospital_audit.log"
DATA_FILE = "hospital_patients.csv"
ANALYTICS_REPORT = "management_report.txt"

# --- 2. LOGGING SETUP (FIXED) ---
# We use force=True to ensure logs are written immediately every time you run the script
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    force=True
)

# --- 3. CLINICAL DATA (MEDICINES & LAB) ---

class Medicine:
    def __init__(self, name, price):
        self.name = name
        self.price = price

class Inventory:
    def __init__(self):
        # Indian Context Medicines & Prices (INR)
        self.med_db = {
            "General": [
                Medicine("Paracetamol", 30), Medicine("Azithromycin", 120),
                Medicine("Cough Syrup", 90), Medicine("Vitamin C", 50)
            ],
            "Cardiology": [
                Medicine("Ecosprin", 60), Medicine("Atorvastatin", 210),
                Medicine("Metoprolol", 140)
            ],
            "Dermatology": [
                Medicine("Ketomac Cream", 180), Medicine("Aloe Vera", 250),
                Medicine("Antifungal Pill", 40)
            ],
            "Orthopedics": [
                Medicine("Calcium D3", 150), Medicine("Pain Relief Gel", 160),
                Medicine("Multivitamin", 100)
            ]
        }

    def get_meds(self, dept):
        """Returns 2 random medicines and their total cost."""
        options = self.med_db.get(dept, self.med_db["General"])
        selected = random.sample(options, k=2)
        cost = sum(m.price for m in selected)
        med_names = ", ".join([m.name for m in selected])
        return med_names, cost

    def get_lab_report(self):
        """Generates random vitals."""
        sugar = random.randint(75, 180)
        hemo = round(random.uniform(10, 16), 1)
        status = "Normal" if sugar < 140 else "Observation Needed"
        return f"Sugar:{sugar}mg/dL, Hb:{hemo}g/dL", status

# --- 4. CORE CLASSES ---

class Doctor:
    def __init__(self, name, dept, fee):
        self.name = name
        self.dept = dept
        self.fee = fee

class Patient:
    def __init__(self, name, age, symptom):
        self.id = random.randint(1001, 9999)
        self.name = name
        self.age = age
        self.symptom = symptom
        self.doctor = None
        self.meds_str = ""
        self.lab_str = ""
        self.lab_status = ""
        self.bill = 0

# --- 5. HOSPITAL SYSTEM ---

class HospitalSystem:
    def __init__(self):
        self.inventory = Inventory()
        self.doctors = {
            "General": Doctor("Dr. Sharma", "General", 500),
            "Cardiology": Doctor("Dr. Verma", "Cardiology", 1500),
            "Dermatology": Doctor("Dr. Iyer", "Dermatology", 800),
            "Orthopedics": Doctor("Dr. Singh", "Orthopedics", 1000)
        }
        self._init_csv()

    def _init_csv(self):
        """Initialize the CSV with headers if it doesn't exist."""
        if not os.path.exists(DATA_FILE):
            with open(DATA_FILE, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "PatientID", "Name", "Age", "Symptom", "Department", 
                    "Doctor", "Medicines", "Lab_Report", "Lab_Status", "Total_Bill_INR"
                ])

    def get_dept(self, symptom):
        s = symptom.lower()
        if any(k in s for k in ['heart', 'chest', 'breath']): return "Cardiology"
        if any(k in s for k in ['skin', 'rash', 'itch']): return "Dermatology"
        if any(k in s for k in ['bone', 'pain', 'joint']): return "Orthopedics"
        return "General"

    def process_patient(self, name, age, symptom):
        print(f"\n[System] Processing patient {name}...")
        
        p = Patient(name, age, symptom)
        
        # 1. Assign Doctor
        dept = self.get_dept(symptom)
        p.doctor = self.doctors[dept]
        
        # 2. Get Meds & Lab
        p.meds_str, med_cost = self.inventory.get_meds(dept)
        p.lab_str, p.lab_status = self.inventory.get_lab_report()
        
        # 3. Calculate Bill (Consultation + Meds + Lab Fee ₹500)
        p.bill = p.doctor.fee + med_cost + 500
        
        # 4. Save to CSV
        self.save_to_csv(p)
        
        # 5. Log Event
        logging.info(f"Patient Processed: {p.name} (ID:{p.id}) | Doctor: {p.doctor.name} | Bill: INR {p.bill}")
        
        # 6. Print Invoice
        print(f"---------------------------------")
        print(f"INVOICE FOR: {p.name} (ID: {p.id})")
        print(f"Doctor: {p.doctor.name} ({p.doctor.dept})")
        print(f"Medicines: {p.meds_str}")
        print(f"Lab Report: {p.lab_str} [{p.lab_status}]")
        print(f"TOTAL BILL: ₹{p.bill}")
        print(f"---------------------------------")

    def save_to_csv(self, p):
        """Appends patient data to the single CSV file."""
        with open(DATA_FILE, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                p.id, p.name, p.age, p.symptom, p.doctor.dept, 
                p.doctor.name, p.meds_str, p.lab_str, p.lab_status, p.bill
            ])
        print(f"[Success] Data saved to {DATA_FILE}")

# --- 6. ETL ANALYTICS ---

class ETL:
    def run(self):
        print("\n[ETL] Running Analytics on Patient Data...")
        
        if not os.path.exists(DATA_FILE):
            print("[Error] No data file found. Add patients first.")
            return

        total_rev = 0
        patients = 0
        critical = 0
        dept_counts = {}

        try:
            with open(DATA_FILE, 'r') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    patients += 1
                    bill = float(row['Total_Bill_INR'])
                    total_rev += bill
                    
                    dept = row['Department']
                    dept_counts[dept] = dept_counts.get(dept, 0) + 1
                    
                    if "Observation" in row['Lab_Status']:
                        critical += 1
            
            # Write Report
            with open(ANALYTICS_REPORT, 'w') as f:
                f.write("HOSPITAL MANAGEMENT REPORT\n")
                f.write("==========================\n")
                f.write(f"Total Revenue: ₹{total_rev}\n")
                f.write(f"Total Patients: {patients}\n")
                f.write(f"Critical Cases: {critical}\n")
                f.write("--------------------------\n")
                f.write("PATIENTS PER DEPT:\n")
                for d, c in dept_counts.items():
                    f.write(f"{d}: {c}\n")

            logging.info("ETL Analytics ran successfully.")
            print(f"[ETL] Report generated: {ANALYTICS_REPORT}")
            
            # Show report content
            with open(ANALYTICS_REPORT, 'r') as f:
                print(f.read())

        except Exception as e:
            logging.error(f"ETL Error: {e}")
            print(f"ETL Failed: {e}")

# --- 7. MAIN MENU ---

def main():
    hospital = HospitalSystem()
    etl = ETL()
    
    print("=== SMART HOSPITAL SYSTEM (INR Version) ===")
    
    while True:
        choice = input("\n1. Add Patient\n2. Run ETL Analytics\n3. View Audit Logs\n4. Exit\nChoice: ")
        
        if choice == '1':
            name = input("Patient Name: ")
            if not name: continue
            age = input("Age: ")
            symptom = input("Symptom: ")
            hospital.process_patient(name, age, symptom)
            
        elif choice == '2':
            etl.run()
            
        elif choice == '3':
            print(f"\n--- LOG FILE CONTENT ---")
            if os.path.exists(LOG_FILE):
                with open(LOG_FILE, 'r') as f:
                    print(f.read())
            else:
                print("Log file is empty.")
                
        elif choice == '4':
            print("Exiting...")
            break

if __name__ == "__main__":
    main()

=== SMART HOSPITAL SYSTEM (INR Version) ===

[System] Processing patient JObanb...
[Success] Data saved to hospital_patients.csv
---------------------------------
INVOICE FOR: JObanb (ID: 4452)
Doctor: Dr. Sharma (General)
Medicines: Azithromycin, Vitamin C
Lab Report: Sugar:101mg/dL, Hb:11.3g/dL [Normal]
TOTAL BILL: ₹1170
---------------------------------
Exiting...
